In [76]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import dotenv
import re
import pandas as pd
import plotly.graph_objects as go

# Dataframe
## Dataframe Setup

In [77]:
df_hubs = pd.read_csv('Hub List all time + pick up and drop offs.csv')
df_pickup = pd.read_csv('2024 - Hub ID & Drop Off - Pick Up numbers.csv',sep=";")

In [78]:
df_pickup

,hub_id,Days in full_date,Average bike_count,# of unique dropoff_hub_id,# of unique pickup_hub_id
0,2180.0,01/01/2024,6.282412,NaN,NaN
1,2180.0,02/01/2024,6.080100,NaN,NaN
2,2180.0,04/01/2024,6.078173,NaN,NaN
3,2180.0,05/01/2024,5.936238,NaN,NaN
4,2180.0,07/01/2024,6.443539,NaN,NaN
...,...,...,...,...,...
6925,26889.0,14/09/2024,6.150918,21.0,20.0
6926,26889.0,15/09/2024,6.229018,12.0,12.0
6927,26889.0,16/09/2024,6.177963,11.0,14.0
6928,26889.0,17/09/2024,6.418395,16.0,18.0


In [79]:
df_pickup.rename(columns = {'hub_id':'id'}, inplace=True)
df_combined = pd.merge(df_pickup, df_hubs, on='id')
df_combined["pick_drop_relation"] = round(df_combined["dropoffs"] /df_combined["pickups"],4)
df_combined["pick_drop_deficit"] = df_combined["# of unique dropoff_hub_id"] - df_combined["# of unique pickup_hub_id"]
df_combined['Days in full_date'] = pd.to_datetime(df_combined['Days in full_date'],format='%d/%m/%Y')

In [80]:
print(df_hubs.shape)
print(df_pickup.shape)
print(df_combined.shape)

(233, 7)
(6930, 5)
(6707, 13)


In [81]:
#find discrepency where bikes dont regulate themselves
df_combined.sort_values("pick_drop_relation",ascending=True)
df_combined.groupby(by="name")[['pickups', "dropoffs", "pick_drop_relation", "pick_drop_deficit"]].mean().sort_values("pick_drop_relation", ascending=False)

,pickups,dropoffs,pick_drop_relation,pick_drop_deficit
name,,,,
"Wik, Kanalfähre - Busstation",2929.0,3033.0,1.0355,-0.250000
MEKUN Olympiahochhaus,15389.0,15721.0,1.0216,-0.221374
Wellingdorf Stadtteilzentrum,6035.0,6136.0,1.0167,-0.166667
Alter Markt,15439.0,15676.0,1.0154,-0.307692
Anleger Dietrichsdorf,4627.0,4683.0,1.0121,-0.155378
thyssenkrupp Marine Systems,11136.0,11255.0,1.0107,-0.129412
Hörnbad,9990.0,10091.0,1.0101,-0.231076
Tilsiter Platz,5658.0,5697.0,1.0069,-0.205426
Vinetaplatz,8225.0,8281.0,1.0068,-0.131687


## Dataframe IDA

In [82]:
print(f'The data lists {len(df_combined.id.unique())} unique ids and {len(df_combined.name.unique())} unique names.')
#time series of specific place
sandkrug_df = df_combined[df_combined['name'] == 'Sandkrug'].sort_values("Days in full_date", ascending=False)

The data lists 36 unique ids and 36 unique names.


In [83]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sandkrug_df['Days in full_date'], y=sandkrug_df['pick_drop_deficit'], mode='lines', name='Value'))
fig.update_layout(title='Time Series Line Plot', xaxis_title='Date', yaxis_title='Value')
fig.show()

In [84]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sandkrug_df['Days in full_date'], y=sandkrug_df['# of unique dropoff_hub_id'], mode='lines', name='Dropoff'))
fig.add_trace(go.Scatter(x=sandkrug_df['Days in full_date'], y=sandkrug_df['# of unique pickup_hub_id'], mode='lines', name='Pickup'))
fig.update_layout(title='Time Series Line Plot', xaxis_title='Date', yaxis_title='Value')
fig.show()

In [85]:
Wik_bus_df = df_combined[df_combined['name'] == ' Wik, Kanalfähre - Busstation'].sort_values("Days in full_date", ascending=False)
fig = go.Figure()
fig.add_trace(go.Scatter(x=Wik_bus_df['Days in full_date'], y=Wik_bus_df['# of unique dropoff_hub_id'], mode='lines', name='Dropoff'))
fig.add_trace(go.Scatter(x=Wik_bus_df['Days in full_date'], y=Wik_bus_df['# of unique pickup_hub_id'], mode='lines', name='Pickup'))
fig.update_layout(title='Time Series Line Plot', xaxis_title='Date', yaxis_title='Value')
fig.show()

# API
## API ACCESS

In [86]:
# .env
config = dotenv.dotenv_values("paul_sprotte.env")

PASSWORD = config["PASSWORD"]

CLIENT_SECRET = config["CLIENT_SECRET"]

In [87]:
token_url = 'https://accounts.kielregion.addix.io/realms/infoportal/protocol/openid-connect/token'
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

data = {
    'grant_type': 'password',
    'username': 'business.brodmapa@outlook.com', 
    'password': PASSWORD,
    'client_id': 'quantumleap',
    'client_secret': CLIENT_SECRET
}

response = requests.post(token_url, headers=headers, data=data)

if response.status_code == 200:
    token_data = response.json()
    access_token = token_data['access_token']
    print(f"Bearer Token successful requested")
    if access_token:
        dotenv_path = '.env'
        
        dotenv.set_key(dotenv_path, 'ACCESS_TOKEN', access_token)
        print(f"Access Token erfolgreich in die .env-Datei geschrieben.")
else:
    print(f"Error: {response.status_code}, {response.text}")

Bearer Token successful requested
Access Token erfolgreich in die .env-Datei geschrieben.


In [88]:
ACCESS_TOKEN = config["ACCESS_TOKEN"]

In [89]:
url = "https://apis.kielregion.addix.io/ql/v2/entities/urn:ngsi-ld:BikeHireDockingStation:KielRegion:26889"
params = {
    'type': 'BikeHireDockingStation',
    'fromDate': '2024-09-13T00:00:00',
    'toDate': '2024-09-13T23:59:59',
    'attrs': 'name,totalSlotNumber,availableBikeNumber,freeSlotNumber'
}

headers = {
    'NGSILD-Tenant': 'infoportal',
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    response_data = response.json()
    print('got a response')
else:
    print(f"Error: {response.status_code}, {response.text}")

Error: 401, {"error":"invalid_grant","error_description":"Invalid bearer token"}


In [90]:
print(response_data)

NameError: name 'response_data' is not defined

## Weather

In [92]:
from pprint import pprint

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
from matplotlib import colormaps

from wetterdienst import Settings
from wetterdienst.provider.dwd.observation import (
    DwdObservationDataset,
    DwdObservationParameter,
    DwdObservationPeriod,
    DwdObservationRequest,
    DwdObservationResolution,
)
%matplotlib inline

In [95]:
pprint(DwdObservationRequest.discover())

{'annual': {'cloud_cover_total': {'origin': '1/8', 'si': 'pct'},
            'count_weather_type_dew': {'origin': '-', 'si': '-'},
            'count_weather_type_fog': {'origin': '-', 'si': '-'},
            'count_weather_type_glaze': {'origin': '-', 'si': '-'},
            'count_weather_type_hail': {'origin': '-', 'si': '-'},
            'count_weather_type_sleet': {'origin': '-', 'si': '-'},
            'count_weather_type_storm_stormier_wind': {'origin': '-',
                                                       'si': '-'},
            'count_weather_type_storm_strong_wind': {'origin': '-', 'si': '-'},
            'count_weather_type_thunder': {'origin': '-', 'si': '-'},
            'precipitation_height': {'origin': 'mm', 'si': 'kg / m ** 2'},
            'precipitation_height_max': {'origin': 'mm', 'si': 'kg / m ** 2'},
            'snow_depth': {'origin': 'cm', 'si': 'm'},
            'snow_depth_new': {'origin': 'cm', 'si': 'm'},
            'sunshine_duration': {'origin': '

In [96]:
request = DwdObservationRequest(
    parameter=DwdObservationDataset.PRECIPITATION_MORE,
    resolution=DwdObservationResolution.DAILY,
    period=DwdObservationPeriod.HISTORICAL,
).all()
request.df.head()

station_id,start_date,end_date,latitude,longitude,height,name,state
str,"datetime[μs, UTC]","datetime[μs, UTC]",f64,f64,f64,str,str
"""00001""",1912-01-01 00:00:00 UTC,1986-06-30 00:00:00 UTC,47.8413,8.8493,478.0,"""Aach""","""Baden-Württemberg"""
"""00002""",1951-01-01 00:00:00 UTC,2006-12-31 00:00:00 UTC,50.8066,6.0996,138.0,"""Aachen (Kläranlage)""","""Nordrhein-Westfalen"""
"""00003""",1891-01-01 00:00:00 UTC,2011-03-31 00:00:00 UTC,50.7827,6.0941,202.0,"""Aachen""","""Nordrhein-Westfalen"""
"""00004""",1951-01-01 00:00:00 UTC,1979-10-31 00:00:00 UTC,50.7683,6.1207,243.0,"""Aachen-Brand""","""Nordrhein-Westfalen"""
"""00006""",1982-11-01 00:00:00 UTC,2024-09-29 00:00:00 UTC,48.8361,10.0598,455.0,"""Aalen-Unterrombach""","""Baden-Württemberg"""


In [97]:
request.df.shape

(5740, 8)

In [99]:
values = (
    DwdObservationRequest(
        parameter=DwdObservationDataset.PRECIPITATION_MORE,
        resolution=DwdObservationResolution.DAILY,
        period=DwdObservationPeriod.HISTORICAL,
    )
    #https://www.dwd.de/DE/leistungen/klimadatendeutschland/stationsuebersicht.html
    .filter_by_station_id(station_id=[2564])
    .values.all()
    .df
)
values.drop_nulls().head()

/Users/paulbrodmann/Documents/Studium/Master/Kiel/WiSe2024-25/Application_Project/Project/.venv/lib/python3.12/site-packages/wetterdienst/provider/dwd/observation/parser.py:151: PerformanceWarning:

Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.



station_id,dataset,parameter,date,value,quality
str,str,str,"datetime[μs, UTC]",f64,f64
"""02564""","""precipitation_more""","""precipitation_form""",1986-06-01 00:00:00 UTC,6.0,10.0
"""02564""","""precipitation_more""","""precipitation_form""",1986-06-02 00:00:00 UTC,0.0,10.0
"""02564""","""precipitation_more""","""precipitation_form""",1986-06-03 00:00:00 UTC,6.0,10.0
"""02564""","""precipitation_more""","""precipitation_form""",1986-06-04 00:00:00 UTC,6.0,10.0
"""02564""","""precipitation_more""","""precipitation_form""",1986-06-05 00:00:00 UTC,6.0,10.0
